In [1]:
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import plotly.graph_objects as go
import requests
import sys

sys.path.append('/Users/samuelhmorton/indiv_projects/work/velo-research')
from ratings import utils

In [2]:
with open('../data/eval-data/men-itt-2000-2023-none-prologue-eval.json') as f:
    prologue_df = json.load(f)
    for key in prologue_df:
        prologue_df[key]['date'] = date.fromisoformat(prologue_df[key]['date'])
f.close()
# with open('../data/eval-data/men-itt-2000-2023-flat-none-eval.json') as f:
#     flat_df = json.load(f)
#     for key in flat_df:
#         flat_df[key]['date'] = date.fromisoformat(flat_df[key]['date'])
# f.close()

# with open('../data/eval-data/men-itt-2000-2023-hilly-none-eval.json') as f:
#     hilly_itt_df = json.load(f)
#     for key in hilly_itt_df:
#         hilly_itt_df[key]['date'] = date.fromisoformat(hilly_itt_df[key]['date'])
# f.close()

with open('../data/eval-data/men-itt-2000-2023-none-none-eval.json') as f:
    itt_df = json.load(f)
    for key in itt_df:
        itt_df[key]['date'] = date.fromisoformat(itt_df[key]['date'])
f.close()

ratings = pd.read_csv('../data/men_velodata.csv')
dates = []
for i in range(len(ratings.index)):
    year = int(ratings.year.iloc[i])
    month = int(ratings.month.iloc[i])
    day = int(ratings.day.iloc[i]) if ratings.day.iloc[i] > 0 else 1
    dates.append(date(year = year, month = month, day = day))
    
ratings.insert(5, 'date', dates)
ratings.drop(columns = ['year', 'month', 'day'], inplace = True)
ratings.sort_values(by = 'date', inplace = True)
ratings.head()

,name,stage,points_scale,parcours_type,date,type,length,profile_score,avg_speed,vertical_meters,won_how,place,rider,team,age,time
1430421,tour-down-under,stage-1,2.1.Stage,p0,2000-01-18,standard,52.2,NaN,46.828 km/h,NaN,? - let us know!,1.0,MOERENHOUT Koos,Farm Frites,26.0,0.0
1430491,tour-down-under,stage-1,2.1.Stage,p0,2000-01-18,standard,52.2,NaN,46.828 km/h,NaN,? - let us know!,71.0,MAIGNAN Gilles,AG2R Prevoyance,31.0,0.0
1430489,tour-down-under,stage-1,2.1.Stage,p0,2000-01-18,standard,52.2,NaN,46.828 km/h,NaN,? - let us know!,69.0,ESTADIEU Laurent,AG2R Prevoyance,24.0,0.0
1430488,tour-down-under,stage-1,2.1.Stage,p0,2000-01-18,standard,52.2,NaN,46.828 km/h,NaN,? - let us know!,68.0,BERGÈS Stéphane,BigMat - Auber 93,25.0,0.0
1430487,tour-down-under,stage-1,2.1.Stage,p0,2000-01-18,standard,52.2,NaN,46.828 km/h,NaN,? - let us know!,67.0,DA CRUZ Carlos,BigMat - Auber 93,25.0,0.0


In [12]:
"""
Features:
stage number
race class
mean rating of top 3 entries
mean rating of top 10 entries
diff between top rated rider and 10th rated rider
tt length
tt vert
itt rating
prologue itt rating
flat itt rating
hilly itt rating
gc rating
"""

tt_data = {
    'race_name': [], 'rider': [], 'timegap': [], 'date': [], 'stage_num': [], 'race_class': [], 
    'length': [], 'vert': [], 'top3_mean': [], 'top10_mean': [], 
    'top10_diff': [], 'general_tt_rating': [], 'prologue_tt_rating': [],# 'flat_tt_rating': [], 
    #'hilly_tt_rating': []
}

for key in itt_df:

    if itt_df[key]['date'] > date(year = 2004, month = 1, day = 1) and len(itt_df[key]['top_active']) >= 40:
        
        # get stage number
        stage_num = int(key.split('stage-')[1][0]) if 'stage' in key else 0
        
        # get race class
        race_class = itt_df[key]['points_scale']
        
        # get race length
        race_length = itt_df[key]['length']
        
        # get race vert
        race_vert = itt_df[key]['vert']
        
        top3_mean = np.mean([t[1] for t in itt_df[key]['top_active'][0: 3]])
        top10_mean = np.mean([t[1] for t in itt_df[key]['top_active'][0: 10]])
        top10_diff = itt_df[key]['top_active'][0][1] - itt_df[key]['top_active'][9][1]

        # get ITT ratings for each rider in the result
        for i in range(len(itt_df[key]['top_active'])):
            rider = itt_df[key]['top_active'][i][0]
            general_rating = itt_df[key]['top_active'][i][1]
            prologue_rating = (prologue_df[key]['top_active']
                               [[t[0] for t in prologue_df[key]['top_active']].index(rider)][1]
                              )
#             flat_rating = (flat_df[key]['top_active']
#                                [[t[0] for t in flat_df[key]['top_active']].index(rider)][1]
#                               )
#             hilly_rating = (hilly_df[key]['top_active']
#                                [[t[0] for t in hilly_df[key]['top_active']].index(rider)][1]
#                               )
            timegap = itt_df[key]['timegaps'][i]
        
            tt_data['race'].append()
            tt_data['rider'].append(rider)
            tt_data['timegap'].append(timegap)
            tt_data['date'].append(itt_df[key]['date'])
            tt_data['stage_num'].append(stage_num)
            tt_data['race_class'].append(race_class)
            tt_data['length'].append(race_length)
            tt_data['vert'].append(race_vert)
            tt_data['top3_mean'].append(top3_mean)
            tt_data['top10_mean'].append(top10_mean)
            tt_data['top10_diff'].append(top10_diff)
            tt_data['general_tt_rating'].append(general_rating)
#             tt_data['prologue_tt_rating'].append(prologue_rating)
    #         tt_data['flat_tt_rating'].append(flat_rating)
    #         tt_data['hilly_tt_rating'].append(hilly_rating)

tt_data = pd.DataFrame(tt_data)
tt_data

,rider,timegap,date,stage_num,race_class,length,vert,top3_mean,top10_mean,top10_diff,general_tt_rating,prologue_tt_rating
0,WOHLBERG Eric,0.0,2004-02-10,5,2.1.Stage,18.0,NaN,1508.283223,1504.975795,8.603829,1509.470023,1506.184824
1,MONINGER Scott,13.0,2004-02-10,5,2.1.Stage,18.0,NaN,1508.283223,1504.975795,8.603829,1509.033285,1506.118622
2,LIESWYN John,21.0,2004-02-10,5,2.1.Stage,18.0,NaN,1508.283223,1504.975795,8.603829,1506.346360,1503.370084
3,GREEN Roland,22.0,2004-02-10,5,2.1.Stage,18.0,NaN,1508.283223,1504.975795,8.603829,1506.318217,1506.151708
4,LANCASTER Brett,35.0,2004-02-10,5,2.1.Stage,18.0,NaN,1508.283223,1504.975795,8.603829,1505.485031,1503.226272
...,...,...,...,...,...,...,...,...,...,...,...,...
97106,BARBERO Carlos,27.0,2022-07-02,0,2.1.Stage,2.3,41.0,1522.474767,1509.322386,24.862389,1482.999961,1489.857545
97107,BATTAGLIN Enrico,28.0,2022-07-02,0,2.1.Stage,2.3,41.0,1522.474767,1509.322386,24.862389,1481.608551,1486.933159
97108,VANHOUCKE Harm,31.0,2022-07-02,0,2.1.Stage,2.3,41.0,1522.474767,1509.322386,24.862389,1479.815494,1485.605431
97109,FERNÁNDEZ Rubén,44.0,2022-07-02,0,2.1.Stage,2.3,41.0,1522.474767,1509.322386,24.862389,1479.539442,1485.126114


In [13]:
itt_df.keys()

dict_keys(['tour-de-langkawi-2000-stage-1-(itt)', 'vuelta-a-la-comunidad-valenciana-2000-stage-5b-(itt)', 'paris-nice-2000-prologue', 'paris-nice-2000-stage-6-(itt)', 'vuelta-ciclista-a-la-region-de-murcia-2000-stage-5-(itt)', 'volta-ao-algarve-2000-stage-4-(itt)', 'tirreno-adriatico-2000-stage-5-(itt)', 'g.p.-portugal-telecom-2000-stage-4-(itt)', 'tour-de-normadie-2000-prologue', 'setmana-catalana-2000-stage-5b-(itt)', 'oxyclean-classic-brugge-de-panne-2000-stage-3b-(itt)', 'itzulia-basque-country-2000-stage-5b-(itt)', 'tour-of-the-alps-2000-stage-1b-(itt)', 'volta-ao-alentejo-2000-stage-1-(itt)', 'tour-de-romandie-2000-prologue', 'tour-de-romandie-2000-stage-3b-(itt)', '4-jours-de-dunkerque-2000-stage-6-(itt)', 'vuelta-asturias-2000-prologue', 'giro-d-italia-2000-prologue', 'giro-d-italia-2000-stage-11-(itt)', 'giro-d-italia-2000-stage-20-(itt)', '60th-tour-de-picardie-2000-stage-3b-(itt)', 'course-de-la-paix-2000-stage-4-(itt)', 'bayern-rundfarht-2000-stage-1b-(itt)', 'olympias-tour